In [55]:
import pandas as pd
import numpy as np
import networkx as nx
import os
ROOTPATH = os.getcwd()[:-3]

In [59]:
epinions_edgelist = pd.read_csv(ROOTPATH + "data\\preprocessed\\epinions_edges.csv")
wikipedia_edgelist = pd.read_csv(ROOTPATH + "data\\preprocessed\\wikipedia_edges.csv")

In [60]:
epinions_edgelist.head(3)

,from,to,sign
0,0,1,-1
1,1,128552,-1
2,2,3,1


In [61]:
wikipedia_edgelist.head(3)

,from,to,sign,timestamp
0,1,2,1,1095171960
1,3,2,-1,1095173580
2,4,2,1,1095174480


In [94]:
def receptive_generative_baselines(edgelist_df):
    
    # Note: edgelist_df must have 3 columns: from (edge from), to (edge to) and sign (of edge, -1 or 1)

    generative_type=edgelist_df.groupby(['from','sign']).count().reset_index()
    generative_type.columns=['node','type','count_type']
    generative_total=pd.DataFrame(edgelist_df.groupby('from').count().reset_index())[['from','to']]
    generative_total.columns=['node','count_total']
    generative=generative_type.merge(generative_total,how='left',on='node')
    generative['generative_probability']=generative['count_type']/generative['count_total']
    generative.columns=['node','type','generative_count','generative_count_total','generative_probability']
    generative=generative[['node','type','generative_count','generative_probability']]

    # receptive probability
    receptive_type=edgelist_df.groupby(['to','sign']).count().reset_index()
    receptive_type.columns=['node','type','count_type']
    receptive_total=pd.DataFrame(edgelist_df.groupby('to').count().reset_index())[['to','from']]
    receptive_total.columns=['node','count_total']
    receptive=receptive_type.merge(receptive_total,how='left',on='node')
    receptive['receptive_probability']=receptive['count_type']/receptive['count_total']
    receptive.columns=['node','type','receptive_count','receptive_count_total','receptive_probability']
    receptive=receptive[['node','type','receptive_count','receptive_probability']]

    # merge
    generative_receptive=generative.merge(receptive,how='outer',on=['node','type']).fillna(0).sort_values('node').reset_index(drop=True)

    return generative_receptive

In [95]:
generative_receptive_epinions=receptive_generative_baselines(epinions_edgelist[['from','to','sign']])
generative_receptive_epinions.head(10)

,node,type,generative_count,generative_probability,receptive_count,receptive_probability
0,0,-1,1.0,1.000000,0.0,0.00000
1,1,1,0.0,0.000000,1.0,0.50000
2,1,-1,1.0,1.000000,1.0,0.50000
3,2,1,1.0,1.000000,1.0,1.00000
4,3,1,0.0,0.000000,4.0,1.00000
5,4,-1,5.0,0.357143,0.0,0.00000
6,4,1,9.0,0.642857,1.0,1.00000
7,5,-1,491.0,0.496461,134.0,0.09882
8,5,1,498.0,0.503539,1222.0,0.90118
9,6,-1,2.0,1.000000,0.0,0.00000


In [96]:
generative_receptive_wikipedia=receptive_generative_baselines(wikipedia_edgelist[['from','to','sign']])
generative_receptive_wikipedia

,node,type,generative_count,generative_probability,receptive_count,receptive_probability
0,1,1,20.0,1.000000,27.0,0.900000
1,1,-1,0.0,0.000000,3.0,0.100000
2,2,1,3.0,0.750000,22.0,0.956522
3,2,-1,1.0,0.250000,1.0,0.043478
4,3,1,57.0,0.647727,0.0,0.000000
...,...,...,...,...,...,...
10487,7116,-1,0.0,0.000000,7.0,0.777778
10488,7116,1,0.0,0.000000,2.0,0.222222
10489,7117,-1,0.0,0.000000,2.0,0.500000
10490,7117,1,0.0,0.000000,2.0,0.500000


In [97]:
generative_receptive_wikipedia.to_csv('generative_receptive_wikipedia.csv')
generative_receptive_epinions.to_csv('generative_receptive_epinions.csv')